# Wandb : Pytorch

* Wandb : 데이터를 시각화하는 툴
* tensorboard랑 비슷한 툴

* 0714fddca3d8c2ab50ab0e32cf0de1bdebc97316

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

In [3]:
# pip install wandb

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\jack0\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.



  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=54e3f66f5eebc81486a60795234b6bfda19b43da38bc63267327f4f63c86bb90
  Stored in directory: c:\users\jack0\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8785 sha256=2c9545da399eda043776d7262da4a052a2daaa6e08da4ecd4608b90dd12ebd53
  Stored in directory: c:\users\jack0\appdata\local\pip\cache\wheels\4c\8e\7e\72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise pathtools


In [4]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\jack0/.netrc


True

In [5]:
args = dict(
    total_epochs=3,
    batch_size=128,
    learning_rate=0.005,
    num_classes=10,
    hidden_size=100)

In [6]:
wandb.init(project='fast_campus_deep_learning(Pytorch)') # 프로젝트 이름
wandb.config.update(args) # 선언한 arg로 설정

wandb: Currently logged in as: jack04215. Use `wandb login --relogin` to force relogin


In [7]:
# Data loader
train_loader = DataLoader(train_data, batch_size=args['batch_size'], shuffle=True)
test_loader = DataLoader(test_data, batch_size=args['batch_size'], shuffle=False)

In [8]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        # input size와 output size 모두 넣어주어야 함
        # tensorflow에는 output demension만 넣음
        self.mlp1 = nn.Linear(input_size, hidden_size) # input size와 output size
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
    # tensorflow : call / pytorch : forward    
    def forward(self, x): 
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=args['hidden_size'], num_classes=args['num_classes']).to(device)

In [9]:
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args['learning_rate']) # 최적화 알고리즘 class 선언
wandb.watch(model)

[]

In [13]:
args

{'total_epochs': 3,
 'batch_size': 128,
 'learning_rate': 0.005,
 'num_classes': 10,
 'hidden_size': 100}

* optimizer.zero_grad() : 역전파 단계를 실행하기 전에 변화도(gradient)를 0으로 만듭니다
* loss.backward() : backward로 미분하여 손실함수에 끼친 영향력(변화량)을 구하기 => 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산
* ooptimizer.step() : 손실함수를 최적화하도록 파라미터를 업데이트하는 과정 => 매개변수가 갱신

In [21]:
def train_model(args):
    for epoch in range(args['total_epochs']):
        for i, (images, labels) in enumerate(train_loader):
            images_flatten = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            
            outputs = model(images_flatten)
            ce_loss = CELoss(outputs, labels)
            
            adam_optimizer.zero_grad() # zero_grad()로 .grad 값들을 0으로 초기화시킨 후 학습을 진행
            ce_loss.backward() # 각 파라미터들의 .grad 값에 변화도가 저장
            adam_optimizer.step() # 작동
            
            # 10번마다 한번씩 epoch, ce_loss 출력
            if (i+1)%10 == 0:
                log_dict = {'Current Epoch':epoch, 'CE Loss': ce_loss}
                wandb.log(log_dict)

In [22]:
def test_model():

    with torch.no_grad(): # back propagation 없음
        correct = 0
        total = 0

        for i, (images, labels) in enumerate(test_loader):
            print('i:',i)
            images_flatten = images.reshape(-1, 28*28*1).to(device)
            labels = labels.to(device)

            outputs = model(images_flatten)
            _ , predicted = torch.max(outputs.data, 1) # 최대값 추출
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            log_dict={"test accuracy":correct/total}
            wandb.log(log_dict)

* https://wandb.ai/home 들어가서 확인

In [23]:
train_model(args)

In [24]:
test_model()

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78


In [71]:
def test_model():
    # 학습이 끝난 후 모델 성능 테스트
    # test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해
    
    images_ex = []
    with torch.no_grad(): # gradient 계산하지 않도록 하는 코드
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images_flatten = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images_flatten)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (i+1)%10 == 0: # 10번 iteration을 돌 때 마다
                images_ex.append(wandb.Image(
                    images[0], caption="predicted: {} label: {}".format(predicted[0].item(), labels[0])))
                log_dict={"test accuracy":correct/total}
                wandb.log(log_dict)
                
        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [72]:
test_model()

Accuracy of the network on the 10000 test images: 96.95 %
